Interpolant els resultats d'un llit elàstic, per poder posar a l'èbola la funcio i les dues primeres derivades

Aquest exemple usa els resultats de l'òptim de la Fase 1

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
llit = np.load("Llit_Exemple/Tensor.npy")

In [3]:
llit = llit.reshape((218*154, 3))

## 1.1 Donada la funció avaluada en una mesh, vull la funció pel continu

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
%%time
f = interp2d(llit[:,0], llit[:,1], llit[:,2], kind='cubic')

In [ ]:
from tqdm import tqdm_notebook as t
llit = np.load("Llit_Exemple/Tensor.npy")
M = np.zeros(llit[:,:,0].shape)
for i in t(range(len(M))):
    for j in range(len(M[0])):
        M[i][j] = f(llit[i,j,0], llit[i,j,1])
np.max(M-llit[:,:,2]) #Sanity check, should be small

In [ ]:
N = 500 #Granularitat de la mesh en la que evaluem gradient i hessiana abans d'interpolar

## 1.2 Donada la funció, vull poder evaluar el seu gradient en un punt arbitrari

In [ ]:
f

In [ ]:
f_vec = lambda x: f(*x)

In [ ]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],f_vec,epsilon = 1e-10)

In [ ]:
#friendship with interp2d ended, RectBivariateSpline is my new friend
from scipy.interpolate import RectBivariateSpline
dfx = RectBivariateSpline(X,Y,M[:,:,0])
dfy = RectBivariateSpline(X,Y,M[:,:,1])

In [ ]:
def gradf(x,y):
    return np.array([dfx(x,y)[0], dfy(x,y)[0]]).reshape(2)

In [ ]:
gradf(2,3)

## 1.2 Donada la funció, vull poder evaluar la seva Hessiana en un punt arbitrari

In [ ]:
dfx_vec = lambda x: dfx(*x)
dfy_vec = lambda x: dfy(*x)

In [ ]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],dfx_vec,epsilon = 1e-10)
        
dfxx = RectBivariateSpline(X,Y,M[:,:,0])
dfxy = RectBivariateSpline(X,Y,M[:,:,1])

In [ ]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],dfy_vec,epsilon = 1e-10)
        
dfyx = RectBivariateSpline(X,Y,M[:,:,0])
dfyy = RectBivariateSpline(X,Y,M[:,:,1])

In [ ]:
def Hessf(x,y):
    return np.array([[dfxx(x,y)[0], dfxy(x,y)[0]], [dfyx(x,y)[0], dfyy(x,y)[0]]]).reshape(2,2)

In [ ]:
Hessf(0.2,0.3)

## Ara ens hem de guardar els resultats per poder-li passar les funcions a l'ebola

In [ ]:
import pickle

pickle.dump(f, open("f.p", 'wb'))
pickle.dump(gradf, open("gradf.p", 'wb'))
pickle.dump(Hessf, open("Hessf.p", 'wb'))
